In [6]:
import shutil
import os

import netCDF4 as nc
import pandas as pd
import xarray as xr
import numpy as np

#### Stages to make Latin Hypercube Parameter ensemble

 - Make an ensemble of files
 - Copy one of the SP calibration outputs into each of these file
 - Make a list of variables to modify
 - Create actual hypercube
 - Define the ranges of the parameters using the csv file
 - Apply to parameter list 

#### Control paths

In [7]:
# Averaged SP calibrated values, from paraemter_averaging and paraemeter_merge scripts. 
averaged_SP_params_files = '../fates_params_vertsc_dayl_SPcal.nc' 

# Where to find the list of parameters and their modifications
pft_parameter_list = '../csvfiles_for_parameter_modification/pft_specific_parameter_inputs/PFT_params.csv'

# Adjusted default x calibration file to make a reasonable baseline. 
modified_baseline_param_files='../intermediate_pfiles/modified_average_file2.nc'

# SP parameter files path 
sp_posterior_filepath='../lhc_param_files/'

### Settings for this version of the ensemble

In [8]:
NVERSION=16

# where do we find the parameter ranges? 
parameter_list_file = '../csvfiles_for_parameter_modification/NOCOMP_LHC_parameters.csv'
ppe_root='/glade/u/home/rfisher/rosiefork_fates_global_cal/rosie_ppe'

if NVERSION==1:
    pdirroot='../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_'
    modify_sp=1 
    nlhc_samples = 500
    
if NVERSION==2:
    pdirroot='../paramfiles/NOCOMP_LHC/nocomp_lhc_v2_'
    modify_sp=1
    nlhc_samples = 500
    
if NVERSION==3:
    pdirroot='../paramfiles/NOCOMP_LHC/nocomp_lhc_v3_'
    modify_sp=1
    nlhc_samples = 50

if NVERSION==4:
    pdirroot='../paramfiles/NOCOMP_LHC/nocomp_lhc_v4_'
    modify_sp=0 
    nlhc_samples = 50

if NVERSION==5:
    pdirroot='../paramfiles/NOCOMP_LHC/nocomp_lhc_v5_'
    modify_sp=0 
    nlhc_samples = 301
    parameter_list_file = '../csvfiles_for_parameter_modification/NOCOMP_LHC_parameters_b.csv'

if NVERSION==6:
    pdirroot='../paramfiles/NOCOMP_LHC/nocomp_lhc_v6_'
    modify_sp=1 
    nlhc_samples = 501
    parameter_list_file = '../csvfiles_for_parameter_modification/NOCOMP_LHC_parameters_b.csv'

if NVERSION==7:
    pdirroot='../paramfiles/NOCOMP_LHC/nocomp_lhc_v7_'
    modify_sp=0
    nlhc_samples = 51
    parameter_list_file = '../csvfiles_for_parameter_modification/NOCOMP_LHC_parameters_b.csv'

if NVERSION==8:
    pdirroot='../paramfiles/NOCOMP_LHC/nocomp_lhc_v8_'
    modify_sp=0
    nlhc_samples = 501
    parameter_list_file = '../csvfiles_for_parameter_modification/NOCOMP_LHC_parameters_c.csv'

if NVERSION==10:
    pdirroot='../paramfiles/NOCOMP_LHC/nocomp_lhc_v10_'
    modify_sp=0
    nlhc_samples = 501
    parameter_list_file = '../csvfiles_for_parameter_modification/NOCOMP_LHC_parameters_e.csv'

if NVERSION==11:
    pdirroot='../paramfiles/NOCOMP_LHC/nocomp_lhc_v11_'
    modify_sp=0
    nlhc_samples = 501
    parameter_list_file = '../csvfiles_for_parameter_modification/NOCOMP_LHC_parameters_f.csv'

# 12 is a test ensemble. 
if NVERSION==13:
    pdirroot='../paramfiles/NOCOMP_LHC/nocomp_lhc_v13_'
    modify_sp=0
    nlhc_samples = 101
    parameter_list_file = '../csvfiles_for_parameter_modification/NOCOMP_LHC_parameters_f.csv'

if NVERSION==14:
    pdirroot='../paramfiles/NOCOMP_LHC/nocomp_lhc_v14_'
    modify_sp=0
    nlhc_samples = 101
    parameter_list_file = '../csvfiles_for_parameter_modification/NOCOMP_LHC_parameters_g.csv'
    pout=ppe_root+'/nocomp_parameter_modification_python/paramfiles/SP_LHC_calibrated_files/'
    modified_baseline_param_files=pout+'SP_calib_ens17_n6.nc'

if NVERSION==15:
    pdirroot='../paramfiles/NOCOMP_LHC/nocomp_lhc_v15_'
    modify_sp=0
    nlhc_samples = 101
    parameter_list_file = '../csvfiles_for_parameter_modification/NOCOMP_LHC_parameters_g.csv'
    modified_baseline_param_files = '../../gaussian_emulator/parameter_outputs/SP_LHC_calibration_2_3.nc'

if NVERSION==16: # repeat with a higher baseline resp for BET
    pdirroot='../paramfiles/NOCOMP_LHC/nocomp_lhc_v16_'
    modify_sp=0
    nlhc_samples = 101
    parameter_list_file = '../csvfiles_for_parameter_modification/NOCOMP_LHC_parameters_g.csv'
    modified_baseline_param_files = '../../gaussian_emulator/parameter_outputs/SP_LHC_calibration_2_3.nc'
pfts=range(0,12)

#### Variables to modify in ensemble

In [9]:
if NVERSION==2 or NVERSION==6  or NVERSION==8:
    allvars=[]
    laivars= ['fates_leaf_slatop', 'fates_allom_d2bl1', 'fates_allom_d2ca_coefficient_max', 'fates_leaf_slamax', 'fates_stoich_nitr', 'fates_recruit_seed_supplement', 'fates_q10_mr', 'fates_maintresp_leaf_vert_scaler_coeff2', 'fates_phen_cold_size_threshold', 'fates_turnover_leaf', 'fates_turnover_fnrt', 'fates_alloc_storage_cushion']
    nppvars= ['fates_stoich_nitr', 'fates_leaf_slatop', 'fates_leafn_vert_scaler_coeff2', 'fates_maintresp_leaf_vert_scaler_coeff2', 'fates_q10_mr', 'fates_grperc', 'fates_phen_gddthresh_b', 'fates_phen_gddthresh_b', 'fates_leaf_slamax', 'fates_turnover_leaf', 'fates_maintresp_leaf_vert_scaler_coeff1']
    vegcvars=['fates_stoich_nitr', 'fates_leaf_slatop', 'fates_alloc_storage_cushion', 'fates_maintresp_leaf_vert_scaler_coeff2', 'fates_q10_mr', 'fates_allom_d2bl2', 'fates_turnover_fnrt',  'fates_leafn_vert_scaler_coeff2', 'fates_grperc', 'fates_recruit_seed_supplement', 'fates_recruit_seed_alloc']
    extravars = ['fates_phen_coldtemp','fates_phen_drought_threshold','fates_allom_fnrt_prof_b',
                       'fates_mort_freezetol','fates_phen_mindayson','fates_allom_d2ca_coefficient_min',
                'fates_mort_hf_sm_threshold','fates_mort_scalar_coldstress',
                'fates_mort_scalar_cstarvation','fates_mort_scalar_hydrfailure',
                'fates_rad_leaf_clumping_index']
    
    nocomp_lhc_params = list(set(laivars + nppvars + vegcvars+extravars))

if NVERSION==3 or NVERSION==4:
    nocomp_lhc_params = ['fates_leaf_slatop',
                         'fates_allom_d2ca_coefficient_min',
                         'fates_maintresp_leaf_vert_scaler_coeff2',
                         'fates_stoich_nitr',
                         'fates_rad_leaf_clumping_index']

if NVERSION==5 or NVERSION==7:
    nocomp_lhc_params = ['fates_leaf_slatop',  # 0perc- +20perc
                         'fates_allom_d2ca_coefficient_min', # -50p-+0p 
                         'fates_maintresp_leaf_vert_scaler_coeff2', # -50p-+50p 
                         'fates_stoich_nitr', # -20p-+20p keep high as proxy for all resp. 
                         'fates_rad_leaf_clumping_index', #0.5 - 0.85
                        'fates_allom_d2bl1' , # -50p-+50p
                        'fates_turnover_leaf'] # -20p-+20p

if NVERSION==9 :
     nocomp_lhc_params = ['fates_leaf_slatop', 'fates_leaf_slamax',
          'fates_allom_d2bl1', 'fates_allom_d2ca_coefficient_max', 
          'fates_allom_d2ca_coefficient_min', 'fates_stoich_nitr', 
          'fates_recruit_seed_supplement', 'fates_turnover_leaf',
          'fates_q10_mr', 'fates_maintresp_leaf_vert_scaler_coeff2', 
           'fates_grperc', 'fates_phen_gddthresh_b' ]

if NVERSION==10 :
     nocomp_lhc_params = ['fates_leaf_slatop','fates_allom_l2fr'
          'fates_allom_d2bl1', 'fates_allom_d2ca_coefficient_max', 
          'fates_allom_d2ca_coefficient_min', 'fates_stoich_nitr', 
          'fates_recruit_seed_supplement', 'fates_turnover_leaf',
          'fates_q10_mr', 'fates_maintresp_leaf_vert_scaler_coeff2', 
           'fates_grperc', 'fates_phen_gddthresh_b' ]

if NVERSION==11 :
     nocomp_lhc_params = ['fates_leaf_slatop','fates_allom_l2fr',
          'fates_allom_d2bl1', 'fates_allom_d2ca_coefficient_max', 
          'fates_allom_d2ca_coefficient_min', 'fates_stoich_nitr', 
          'fates_recruit_seed_supplement', 'fates_turnover_leaf',
          'fates_q10_mr', 'fates_maintresp_leaf_vert_scaler_coeff2', 
           'fates_grperc', 'fates_phen_gddthresh_b' ]

if NVERSION==13 or NVERSION==14:
     nocomp_lhc_params = [
          'fates_leaf_slatop',
          'fates_allom_d2bl1', 
          'fates_allom_d2ca_coefficient_min',
           'fates_turnover_leaf',
           'fates_maintresp_leaf_vert_scaler_coeff2', 
           ]
if NVERSION==15 :
     nocomp_lhc_params = [
          'fates_leaf_slatop',
          'fates_allom_d2bl1', 
          'fates_allom_d2ca_coefficient_min',
           'fates_turnover_leaf',
           'fates_maintresp_leaf_vert_scaler_coeff2', 
           'fates_stoich_nitr',
           'fates_recruit_seed_supplement']

if NVERSION==16 :
     nocomp_lhc_params = [
          'fates_leaf_slatop',
          'fates_allom_d2bl1', 
          'fates_allom_d2ca_coefficient_min',
           'fates_turnover_leaf',
           'fates_maintresp_leaf_vert_scaler_coeff2', 
           'fates_stoich_nitr',
           'fates_maintresp_leaf_atkin2017_baserate'
           ]

print(NVERSION,nocomp_lhc_params)
print(len(nocomp_lhc_params))


16 ['fates_leaf_slatop', 'fates_allom_d2bl1', 'fates_allom_d2ca_coefficient_min', 'fates_turnover_leaf', 'fates_maintresp_leaf_vert_scaler_coeff2', 'fates_stoich_nitr', 'fates_maintresp_leaf_atkin2017_baserate']
7


#### Read the main parameter list CSV file into a pandas DataFrame

In [10]:
df = pd.read_csv(parameter_list_file)
fates_parameter_list = df['fates_parameter'].tolist()
minp = df['min'].tolist() 
maxp = df['max'].tolist() 
dimp = df['dim'].tolist()

#### Make a dictionary to index the ranges searchable by parameter name

In [11]:
pftname_vs_csvfileindex = {
    "fates_rad_stem_rhovis": "rhosvis",
    "fates_rad_stem_rhonir": "rhosnir",
    "fates_rad_leaf_rhonir": "rholnir",
    "fates_rad_leaf_rhovis": "rholvis",
    "fates_rad_leaf_taunir": "taulnir",
    "fates_rad_leaf_tauvis": "taulvis",
    "fates_leaf_stomatal_slope_medlyn": "medlynslope",
    "fates_leaf_slatop": "sla",
    "fates_rad_leaf_xl": "xl", 
    "fates_turnover_leaf": "leaf_lifespan"   
}

In [12]:
# index the order of the parameters in the csv input file. 
my_dict = {}
print(len(fates_parameter_list))
for i in range(len(fates_parameter_list)):
    my_dict[fates_parameter_list[i]] = i


121


#### Read the PFT parameter list CSV file into a pandas DataFrame

In [13]:
dfp = pd.read_csv(pft_parameter_list)
pftindex = dfp['pft_index'].tolist()
pftparam = dfp['param'].tolist() 
pftmin = dfp['min'].tolist()
pftmax = dfp['max'].tolist()



#### Map from the variables in the pftvariable csv file into the fates pft space

In [14]:
string_mapping = {
    "fates_rad_stem_rhovis": "rhosvis",
    "fates_rad_stem_rhonir": "rhosnir",
    "fates_rad_leaf_rhonir": "rholnir",
    "fates_rad_leaf_rhovis": "rholvis",
    "fates_rad_leaf_taunir": "taulnir",
    "fates_rad_leaf_tauvis": "taulvis",
    "fates_leaf_stomatal_slope_medlyn": "medlynslope",
    "fates_leaf_slatop": "sla",
    "fates_rad_leaf_xl": "xl",
    "fates_turnover_leaf": "leaf_lifespan"  }

In [15]:
# index the order of the pfts in the input file. 
my_dict_pft = {}
print(len(pftparam))
for j in range(1,len(pftparam)):
    my_dict_pft[pftparam[j]] = j
print(my_dict_pft)

96
{'taulnir': 11, 'taulvis': 23, 'rholnir': 35, 'rholvis': 47, 'xl': 59, 'medlynslope': 71, 'sla': 83, 'leaf_lifespan': 95}


#### Retreive the full list of the parameters from the csv file 

In [16]:
reference_vars = nc.Dataset(modified_baseline_param_files, 'r+') 


#### Make a latin hypercube

In [17]:
num_variables = len(nocomp_lhc_params)

# Generate a Latin Hypercube
hypercube = np.zeros((nlhc_samples, num_variables))

for i in range(num_variables):
    hypercube[:, i] = np.random.permutation(np.linspace(0, 1, nlhc_samples))


#### Modify param files with nocomp modifications in LHC

In [18]:
print('nlhc',nocomp_lhc_params)
for i in range(1,nlhc_samples):
    print('i',i)
    # Make a new filw for each ensemble member
    counter = 0
    newfile = pdirroot + str(i) + '.nc' 
    !rm $newfile
    shutil.copy(modified_baseline_param_files, newfile)
    nc_file = nc.Dataset(newfile, 'r+')
    min_range_pft=np.zeros(12)
    max_range_pft=np.zeros(12)    
    # Perform operations on each target lhc parameter  
    for index, parameter in enumerate(nocomp_lhc_params):
       # print('param',parameter)
        if parameter in reference_vars.variables:
            variable_to_modify = nc_file.variables[parameter]
            csvindex=my_dict[parameter]
            # Find the range for each parameter depending on its type. 
            if(variable_to_modify.ndim==1):
                if((variable_to_modify.size)>2):
                    variable_data = variable_to_modify[:]
                else:
                    variable_data = variable_to_modify[:]           
            elif variable_to_modify.ndim==0:
                variable_data = variable_to_modify[:]
            else:
                variable_data = variable_to_modify[:,:]              
            variable_data=np.squeeze(variable_data)
            strp=0
            
            if minp[csvindex] == '100percent': 
                perc_ch_mn = 0.0
                strp=1
            if minp[csvindex] == '80percent': 
                perc_ch_mn = 0.2
                strp=1               
            if minp[csvindex] == '50percent':  
                perc_ch_mn = 0.5
                strp=1
            if minp[csvindex] == '30percent': 
                perc_ch_mn = 0.7
                strp=1
            if minp[csvindex] == '10percent': 
                perc_ch_mn = 0.9
                strp=1
            if minp[csvindex] == '0percent': 
                perc_ch_mn = 1.0
                strp=1
                
 
            if maxp[csvindex] == '100percent': 
                perc_ch_mx = 2.0
            if maxp[csvindex] == '50percent': 
                perc_ch_mx = 1.8
            if maxp[csvindex] == '80percent': 
                perc_ch_mx = 1.5
            if maxp[csvindex] == '30percent': 
                perc_ch_mx = 1.3
            if maxp[csvindex] == '10percent': 
                perc_ch_mx = 1.1
            if maxp[csvindex] == '0percent': 
                perc_ch_mx = 1.0
             # direct numbers    
            if maxp[csvindex] == 'pft': 
                strp=1
            
            vdtype=variable_to_modify.dtype
            if( strp==1):

                if not minp[csvindex] ==  'pft' :
                    if not vdtype == int:
                        min_range = variable_data * float(perc_ch_mn)
                        max_range = variable_data * float(perc_ch_mx)
                    else: # integer 
                        min_range = (variable_data * perc_ch_mn).astype(int)
                        max_range = (variable_data * perc_ch_mx).astype(int)
                    
                else: #pft 
                    shortvar = string_mapping[parameter]
                    table = dfp[dfp['param'] == shortvar]
                    pftmins=table['min'].tolist()
                    pftmaxs=table['max'].tolist()
                    variable_to_modify[:] *=  0
                    for p in pfts: 
                        min_range_pft[p]  = variable_data[p-1]*0 +float(float(pftmins[p]))
                        max_range_pft[p]  = variable_data[p-1]*0 +float(float(pftmaxs[p]))
            else:
                min_range = variable_data * 0 +  float(minp[csvindex])
                max_range = variable_data * 0 +  float(maxp[csvindex])
     
#-------------------------------------------------------------
# Change variable according to LHC value
#-------------------------------------------------------------
            #print('hypercube',hypercube[i, index],perc_ch_mn,perc_ch_mx)
            if not minp[csvindex] ==  'pft': 
            # Modify variables according to the LHC ranges
                oldp=variable_to_modify
                rangev = max_range-min_range
                hrangev = rangev* hypercube[i, index]
                newvalue = min_range + hrangev
                variable_to_modify = newvalue

                nc_file.variables[parameter][:] = variable_to_modify
                
            else: 
                oldp=variable_to_modify
                for p in range(0, 12 ): 
                    rangep = max_range_pft[p]-min_range_pft[p]
                    hrange = rangep* hypercube[i, index]
                    newvalue = min_range_pft[p] + hrange
                    variable_to_modify[:,p] = newvalue 
                    #print('newvar',newvalue,min_range_pft[p],max_range_pft[p],hrange)
                nc_file.variables[parameter][:] = variable_to_modify[:]
               # print('nv',variable_to_modify[:,:])
        else: #doenst exist 
            print('target variable is not in csv file', parameter)
    nc_file.close()

nlhc ['fates_leaf_slatop', 'fates_allom_d2bl1', 'fates_allom_d2ca_coefficient_min', 'fates_turnover_leaf', 'fates_maintresp_leaf_vert_scaler_coeff2', 'fates_stoich_nitr', 'fates_maintresp_leaf_atkin2017_baserate']
i 1
rm: cannot remove '../paramfiles/NOCOMP_LHC/nocomp_lhc_v16_1.nc': No such file or directory
i 2
rm: cannot remove '../paramfiles/NOCOMP_LHC/nocomp_lhc_v16_2.nc': No such file or directory
i 3
rm: cannot remove '../paramfiles/NOCOMP_LHC/nocomp_lhc_v16_3.nc': No such file or directory
i 4
rm: cannot remove '../paramfiles/NOCOMP_LHC/nocomp_lhc_v16_4.nc': No such file or directory
i 5
rm: cannot remove '../paramfiles/NOCOMP_LHC/nocomp_lhc_v16_5.nc': No such file or directory
i 6
rm: cannot remove '../paramfiles/NOCOMP_LHC/nocomp_lhc_v16_6.nc': No such file or directory
i 7
rm: cannot remove '../paramfiles/NOCOMP_LHC/nocomp_lhc_v16_7.nc': No such file or directory
i 8
rm: cannot remove '../paramfiles/NOCOMP_LHC/nocomp_lhc_v16_8.nc': No such file or directory
i 9
rm: cannot rem

#### check diffs between nocomp lhc members after NOCOMP parameter modification

In [19]:
newfile = pdirroot + str(1) + '.nc' 
nc_file_1 = nc.Dataset(newfile, 'r+')

for i in range(1,nlhc_samples):
    newfile = pdirroot + str(i) + '.nc' 
    nc_file_i = nc.Dataset(newfile, 'r+')

    for var_name in nc_file_i.variables:
        file1_var = nc_file_1.variables[var_name]
        filei_var = nc_file_i.variables[var_name]
        #print(var_name,'ndim',average_var.ndim)
        if file1_var.ndim<2 :
            file1_var=file1_var[:12]
            filei_var=filei_var[:12]
            diffs = abs(filei_var-file1_var)
            if np.size((diffs))>1 :
                sumd=np.sum(diffs[:])
            else:
                sumd=diffs
        else:
            file1_var=file1_var[:12]
            filei_var=filei_var[:12]
            sumd=np.sum(np.sum(diffs[:]))

            
            printdiffs=1  
            if sumd>0.001 :
                print(i, f"Variable {var_name} is different")
                if printdiffs==1:
                    if np.size((diffs))>1 :
                        ln=4
                        print('file_i:',filei_var[1:ln])
                        print('file_1:', file1_var[1:ln])
                        print('diffs:', diffs[1:ln])
                    else:
                        print('file_i:',filei_var)
                        print('file_1:', file1_var)
                        print('diffs:', diffs)

 

#### Find and read the SP parameter files from the last ensemble fitting process (IF we are varying the SP parameters as well)

In [20]:
if(modify_sp==1):
    file_list = os.listdir(sp_posterior_filepath)
    sp_posterior=[]
    for file in file_list:
        file_path = os.path.join(sp_posterior_filepath, file)
        dataset = xr.open_dataset(file_path)
        sp_posterior.append(dataset)
    

#### check diffs between SP lhc members

In [21]:
directory_path = '..//lhc_param_files'

file='FATES_LH_347.nc'
file_path = os.path.join(directory_path, file)
data1 = xr.open_dataset(file_path)
file='FATES_LH_348.nc'
file_path = os.path.join(directory_path, file)
data2 = xr.open_dataset(file_path)

common_variables = set(data1.variables) & set(data2.variables)

# Iterate through the common variables and compare their values
parlist=[]
for var_name in common_variables:
    var_data1 = data1[var_name]
    var_data2 = data2[var_name]
    # Compare the values of the variables
    if not var_data1.equals(var_data2):
        parlist.append(var_name)
print(parlist)

['fates_leaf_vcmax25top', 'fates_leaf_vcmaxse', 'fates_leaf_stomatal_slope_ballberry', 'fates_leaf_stomatal_intercept', 'fates_maintresp_leaf_atkin2017_baserate', 'fates_leaf_vcmaxhd', 'fates_nonhydro_smpsc', 'fates_leaf_jmaxse', 'fates_leaf_vcmaxha']


#### Modify SP variables in the LHC ensemble

In [22]:
if modify_sp ==1:
    for i in range(1,nlhc_samples):
        # Make a new filw for each ensemble member
        counter = 0
        # Open the right LHC parameter file
        newfile = pdirroot + str(i) + '.nc' 
        nc_file = nc.Dataset(newfile, 'r+')
        print(newfile)
        #Find an SP parameter set to use 
        
        # Perform operations on each target lhc parameter  
        for index, parameter in enumerate(parlist):
            base_var = nc_file.variables[parameter]
            sp_lhc_var = sp_posterior[i][parameter].values
            
            #print('av',average_var)
            #print('bs',base_var)
            if(base_var.ndim==1):
                base_var[0:12] = sp_lhc_var[0:12]  
            else:
                base_var[:,0:12] = sp_lhc_var[:,0:12]             
        nc_file.close()
                

#### check diffs between lhc members after SP modification

In [23]:
newfile = pdirroot + str(1) + '.nc' 
nc_file_1 = nc.Dataset(newfile, 'r+')

for i in range(1,nlhc_samples):
    newfile = pdirroot + str(i) + '.nc' 
    nc_file_i = nc.Dataset(newfile, 'r+')

    for var_name in nc_file_i.variables:
        file1_var = nc_file_1.variables[var_name]
        filei_var = nc_file_i.variables[var_name]
        
        if file1_var.ndim<2 :
            file1_var=file1_var[:12]
            filei_var=filei_var[:12]
            diffs = abs(filei_var-file1_var)
            if np.size((diffs))>1 :
                sumd=np.sum(diffs[:])
            else:
                sumd=diffs
                
            printdiffs=1                
            if sumd>0.001:
                print(i, f"Variable {var_name} is different")
                if(np.size((diffs))>1):
                    ln=4
                    print('file_i:',filei_var[1:ln])
                    print('file_1:', file1_var[1:ln])
                    print('diffs:', diffs[1:ln])
                else:
                    print('file_i:',filei_var)
                    print('file_1:', file1_var)
                    print('diffs:', diffs)

 

2 Variable fates_allom_d2bl1 is different
file_i: [0.08883 0.08883 0.01269]
file_1: [0.08568 0.08568 0.01224]
diffs: [0.00315 0.00315 0.00045000000000000075]
2 Variable fates_allom_d2ca_coefficient_min is different
file_i: [0.09751416 0.28478364 0.01529344]
file_1: [0.36050688 1.05283648 0.05653939]
diffs: [0.26299272160000003 0.7680528407999999 0.0412459464]
2 Variable fates_leaf_slatop is different
file_i: [0.00494  0.023712 0.008892]
file_1: [0.0055 0.0264 0.0099]
diffs: [0.0005599999999999997 0.0026879999999999994 0.0010079999999999985]
2 Variable fates_maintresp_leaf_atkin2017_baserate is different
file_i: [1.773 1.773 1.773]
file_1: [1.777 1.777 1.777]
diffs: [0.0040000000000000036 0.0040000000000000036 0.0040000000000000036]
2 Variable fates_maintresp_leaf_vert_scaler_coeff2 is different
file_i: [1.53576 1.53576 1.53576]
file_1: [2.27448 2.27448 2.27448]
diffs: [0.7387200000000003 0.7387200000000003 0.7387200000000003]
3 Variable fates_allom_d2bl1 is different
file_i: [0.08442 0